## Filters

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage

Filterは原資産と時系列データから真偽値を返す関数です。

```
F(asset, timestamp) -> boolean
```

フィルタは銘柄の絞り込みやpipelineの結果を絞り込むために使われます。フィルタを作成するためには比較演算子を用いる方法と Factor/Classifier メソッドを用いる方法があります。

In [2]:
last_close_price = USEquityPricing.close.latest
close_price_filter = last_close_price > 20

下記の例では10日間の移動平均が30日間の移動平均を下回った場合に `True` を返します。

In [3]:
mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)
mean_crossover_filter = mean_close_10 < mean_close_30

Remember, each security will get its own `True` or `False` value each day.

銘柄ごと、日付ごとに異なる真偽値を持っていることに注意してください。

### Factor/Classifier Methods

`Factor` クラスと `Classifier` クラスのさまざまなメソッドは、`Filters` を返します。今回は `Factor` クラスのメソッドを用います(`Classifier` クラスについてはまだ触れていないため後ほど解説します)。`Factor.top(n)` メソッドは、特定のFactorの上位n個の銘柄に対して `True` を返すFilterを生成します。 次の例では、毎日上位200個の銘柄に対して `True` を返すフィルタを作成します。これらの銘柄がすべての銘柄の直近の終値で上位200に入ったことを示しています。

In [4]:
last_close_price = USEquityPricing.close.latest
top_close_price_filter = last_close_price.top(200)

* `Factor` クラスのリファレンス [this link](https://www.quantopian.com/help#quantopian_pipeline_factors_Factor).
* `Classifier` クラスのリファレンス [this link](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier).

## Dollar Volume Filter

30日の売買代金の平均が10,000,000ドルを超えた場合に `True` を返すフィルタを作成してみます。組み込みFactorの `AverageDollarVolume` を用います。

In [5]:
from quantopian.pipeline.factors import AverageDollarVolume

In [6]:
dollar_volume = AverageDollarVolume(window_length=30)

`AverageDollarVolume` はデフォルトで キーワード引数 `inputs` に `USEquityPricing.close` と `USEquityPricing.volume` が渡されるため省略できます。

売買代金が10,000,000ドルを超える銘柄を抽出します。

In [7]:
high_dollar_volume = (dollar_volume > 10000000)

pipelineに追加して中身を確認してみます。

In [8]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30
    
    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = (dollar_volume > 10000000)

    return Pipeline(
        columns={
            'percent_difference': percent_difference,
            'high_dollar_volume': high_dollar_volume
        }
    )

piplineを実行すると、'high_dollar_volume' 列に filter の真偽値が格納されます。

In [9]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

high_dollar_volume  \
2015-05-05 00:00:00+00:00 Equity(2 [ARNC])                   True   
                          Equity(21 [AAME])                 False   
                          Equity(24 [AAPL])                  True   
                          Equity(25 [ARNC_PR])              False   
                          Equity(31 [ABAX])                 False   

                                                percent_difference  
2015-05-05 00:00:00+00:00 Equity(2 [ARNC])                0.017975  
                          Equity(21 [AAME])              -0.002325  
                          Equity(24 [AAPL])               0.016905  
                          Equity(25 [ARNC_PR])            0.021544  
                          Equity(31 [ABAX])              -0.019639

## Applying a Screen

パイプラインはデフォルトですべての銘柄に対して計算が実行されます。通常は特定の条件に合った銘柄のみ計算できれば問題ありません。フィルタ結果が `False` となる銘柄を計算対象から除外することができます。`Pipeline()` 関数のキーワード引数 `screen` に filter を指定します。

In [10]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume > 10000000

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=high_dollar_volume
    )

上記を実行すると high_dollar_volume の条件に合った銘柄のみが対象となります。2015年5月5日を指定して実行した場合には約2,100個の銘柄が対象となります。

In [11]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print 'Number of securities that passed the filter: %d' % len(result)
result.head()

Number of securities that passed the filter: 2106


percent_difference
2015-05-05 00:00:00+00:00 Equity(2 [ARNC])             0.017975
                          Equity(24 [AAPL])            0.016905
                          Equity(53 [ABMD])           -0.024682
                          Equity(62 [ABT])             0.014385
                          Equity(64 [ABX])             0.046963

## Inverting a Filter

~演算子は真偽値を反転させます。これまでと反対に売買代金が低い銘柄を抽出する場合には下記のように記載します。

In [12]:
low_dollar_volume = ~high_dollar_volume

次のレッスンでは fitler を組み合わせを実施します。